# CRIM Intervals:  Cadences

### Reminders:

#### Import Music Files

* If you are exploring pieces from CRIM, importing simply involves providing the CRIM URL of the MEI file:  
    * **`piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')`**

* But you can also use the Notebook with any MEI, MusicXML, or MIDI file of your own. You can easily do this when you run the Notebooks on Jupyter Hub, you will also find a folder called **`Music_Files`**.  Upload the file here, then provide the path to that file: 
    * **`piece = importScore('Music_Files/My_File_Name.mei')`**.  

#### Save outputs as CSV or Excel

* The Jupyter Hub version of these Notebooks also provides a folder called **`saved_csv`**.  You can save **csv** files of any data frame there with this command: 
    * **`notebook_data_frame_name.to_csv('saved_csv/your_file_title.csv')`**.
* If you prefer **Excel** documents (which are better for anything with a complex set of columns or hierarhical index), use **ExcelWriter**.  In the following code, you will need to provide these commands:
    * **`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')`**
* Now convert your dataframe to Excel
    * **`frame_name.to_excel(writer, sheet_name='Sheet1')`**
* And finally save the new file to the folder here in the Notebook:
    * **`writer.save()`**

Put the following code to a new cell and update the frame_name and file_name:

`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')` <br>
`frame_name.to_excel(writer, sheet_name='Sheet1')` <br>
`writer.save()` <br>


## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [15]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece here
mei_file = 'CRIM_Mass_0004_3.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)


Memoized piece detected.
{'title': 'Missa Virginis Mariae: Credo', 'composer': 'Pierre Clereau'}


## C. Find Cadences with Modular Analysis

Alex Morgan has built a powerful tool that identifies cadences according to the combinations of two-voice **modules** that describe the typical contrapuntal motion between the various **cadential voice functions** (**CVF**) heard in Renaissance polyphony:  **cantizans and tenorizans**, **cantizans and bassiszans**, etc.

The tool uses modular analysis to identify **conjunctions** of these pairs in order to predict cadences of various kinds. But there are many combinations, especially once we consider that voices functions (or roles) can be **displaced** (as when the tenorizans role appears in the Superius part and the cantizans appears in the Tenor part), or through **irregular** motion, and even **interrupted**, as when a voice is suddenly silent. 

You can in fact check all of the cadential voice functions (CVFs) for a given piece below.  But Alex's system also conveniently **labels** the cadences according to **type**, **tone**, **evaded** and also provides information about the relative place within the piece, the adjacent cadences, and many other features, too.

Note:  **Measure** and **Beat** columns are in the body of the table, not at the Index.

**Column Headings Explained**:

* The **Low** and **Tone** columns give the pitches of the **lowest sounding pitch (in any voice) at the perfection**, and the **goal tone of the cantizans** (or altizans if there is no cantizans) respectively.

* **RelLow** is the lowest pitch of each cadence shown as an interval measured against the last pitch in the **Low** column. Likewise, **RelTone** is the cadential tone shown as an interval measured against the last pitch in the **Tone** column.

* The **SinceLast** and **ToNext** columns are the time in quarter notes since the last or to the next cadence.

* The **Progress** column is a relative indication of position in the piece.  **0** is the beginning of the piece; **1.0** is the end of the piece.



Read more via the documentation: **`print(piece.classifyCadences.__doc__)`**

View the **Cadential Voice Function** and **Cadence Label** tables here:  https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/.  These can easily be updated with revised or new cadence types.

### C.1 Classify the Cadences

In [16]:
cadences = piece.classifyCadences()
col_list = ['Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
cadences = cadences[col_list]
cadences

ValueError: asof requires a sorted index

### C.2  Check Voice Functions for all Cadences of a Given Piece

When **return_type** is set to **'functions'** (or just 'f' for short), a table
of the **cadential voice functions** (CVF) is returned. Each CVF is
represented with a single-character label with the meanings as follows:

* `C`: **cantizans motion up a step** (can also be ornamented e.g. Landini)
* `T`: **tenorizans motion down a step** (can be ornamented with anticipations)
* `B`: **bassizans motion up a fourth or down a fifth**
* `A`: **altizans motion, similar to cantizans**, but cadences to a fifth
above a tenorizans instead of an octave  <br><br>
* `L`: **leaping contratenor** motion up an octave at the perfection
* `P`: **plagal bassizans** motion up a fifth or down a fourth <br><br>
* `c`: **evaded cantizans** when it moves to an unexpected note at the perfection
* `t`: **evaded tenorizans** when it goes up by step at the perfection
* `b`: **evaded bassizans** when it goes up by step at the perfection
* `u`: **evaded bassizans** when it goes down by third at the perfection
(there are no evaded labels for the altizans, plagal bassizans leaping
contratenor CVFs) <br><br>
* `x`: evaded **bassizans motion where the voice drops out** at the perfection
* `y`: evaded **cantizans motion where the voice drops out** at the perfection
* `z`: evaded **tenorizans motion where the voice drops out** at the perfection


View the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

In [5]:
cvf = piece.classifyCadences(return_type='f')
piece.detailIndex(cvf, offset=True).fillna('-')


,,,[Superius],Altus,Tenor,Bassus
Measure,Beat,Offset,,,,
24,3.0,188.0,-,T,C,b
29,2.0,226.0,-,t,C,-
30,1.0,232.0,C,-,T,b
35,1.0,272.0,C,T,-,-
39,1.0,304.0,-,A,C,T
53,1.0,416.0,C,-,T,B
60,1.0,472.0,C,T,B,-
65,1.0,512.0,-,-,C,T
77,1.0,608.0,C,-,T,B


### C.3.  Check for Missed Cadences (One or More Pieces)


* The resulting table lists the **Cadential Voice Functions** (CVF's) for each of the probable cadences that the Cadence Classifier was **unable to categorize** according to the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

* See below for additional instructions.

In [7]:

corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei']) # add as many pieces as you want here, each in single quotations, separated by commas
cad_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=False)
cvf_dfs = corpus.batch(ImportedPiece.classifyCadences, kwargs={'return_type': 'f'})
missed = []
for i, cad in enumerate(cad_dfs):
    df = cvf_dfs[i].loc[cad.CadType.isnull(), :]
    df = df[df.columns[range(-2, len(df.columns) -2)]]
    df.columns = range(len(df.columns))
    missed.append(df)
result = pd.concat(missed)
rm = piece.detailIndex(result, offset=True)
result_offset_list = rm.index.get_level_values('Offset')
rm

Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei


,,,0,1,2,3,4,5
Measure,Beat,Offset,,,,,,


### C.4.  Check the Harmonic Modules of Missing Cadences

* For a given piece (since this tool works for one composition at a time), you can check the **harmonic modules** (for example `7_Held, 6_-2, 8` for a cantizans-tenorizans pair) at any given point in your piece
    * **Import the piece** (just ONE at a time)
    * Run the **"missed cadence finder"** (Section D) above.  You must run this with just ONE piece!
* The default **modular ngram** is 3 events long.  Adjust **`n`** as needed for longer modules
* The default **interval type** is diatonic ('d').  Adjust **`kind`** as needed for chromatic ('c')

Use the results to report the new cadential voice functions and label, as explained above.

In [8]:
piece_har = piece.getHarmonic(kind='d', compound=False)
ngrams = piece.getNgrams(df=piece_har, n=3, how='modules', exclude=['Rest'], offsets='last')
ngrams_filtered = ngrams.loc[result_offset_list,:].fillna("-")
ngrams_filtered

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
Offset,,,,,,


### C.5.  Inventory of Cadences Found:  One or Many Pieces at Once

* Here you can report an inventory of cadences by **type** and **tone** (and **evaded** status)


* To search multiple pieces at once (each returning its own dataframe of results):  enter the urls, separated by commas (each url within **single quotation marks**, and separated from the previous by a **comma** ).  Thus:

* **`corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0019.mei', 'https://crimproject.org/mei/CRIM_Model_0010.mei'])`**

In [10]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei', 'https://crimproject.org/mei/CRIM_Model_0009.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=False)
combined_df = pd.concat(list_of_dfs, ignore_index=False)
summary = combined_df.groupby(['CadType', 'Tone', 'Evaded']).size().reset_index(name='counts')
summary


Memoized piece detected.
Successfully imported https://crimproject.org/mei/CRIM_Model_0009.mei


,CadType,Tone,Evaded,counts
0,Authentic,C,False,4
1,Authentic,C,True,1
2,Authentic,D,False,1
3,Authentic,F,False,1
4,Authentic,G,True,1
5,Clausula Vera,C,False,2
6,Clausula Vera,C,True,1
7,Clausula Vera,D,False,3
8,Clausula Vera,F,False,1
9,Clausula Vera,G,False,4


A simpler summary, without tone or evaded status:

In [13]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=False)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
# Get the number of each type of cadence observed:
cadTypeCounts = combined_df['Tone'].value_counts()
# Get the number of cadences per Beat level:
# cadTypeCounts = combined_df['Beat'].value_counts()

Memoized piece detected.
Memoized piece detected.


In [14]:
cadTypeCounts.to_frame()

,Tone
C,8
G,5
D,4
E,3
F,2
